### 스코어링 전략 

1. 최근 5일 가격 증가 : 일자별 5점
2. 최근 5일 거래량 증가 : 일자별 5점 
3. 최근일 범위 초과 상승 : 범위별 5점 

- https://wikidocs.net/31063

In [19]:
#!pip install pyupbit

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [ ]:
import pyupbit
import pprint

# 조회만 되는 키 
access_key = "HERE'S YOUR KEY"
secret_key = "HERE'S YOUR KEY"


In [ ]:
upbit = pyupbit.Upbit(access_key, secret_key)

In [ ]:
# 과거 데이터
df = pyupbit.get_ohlcv("KRW-MTL")
df.tail(6)

,open,high,low,close,volume,value
2022-04-22 09:00:00,3185.0,3640.0,3030.0,3050.0,5.567833e+07,1.847379e+11
2022-04-23 09:00:00,3050.0,3720.0,3025.0,3475.0,1.238675e+08,4.289823e+11
2022-04-24 09:00:00,3470.0,3655.0,3180.0,3205.0,4.144866e+07,1.433444e+11
2022-04-25 09:00:00,3205.0,3395.0,3095.0,3310.0,5.455096e+07,1.780220e+11
2022-04-26 09:00:00,3305.0,3610.0,3125.0,3250.0,5.153399e+07,1.760463e+11
2022-04-27 09:00:00,3250.0,3445.0,3250.0,3385.0,1.482144e+07,4.987280e+10


코인 스코어링 함수

In [ ]:
def get_coin_score(ticker):

  df = pyupbit.get_ohlcv(ticker)
  h = 21   # 현재시간 할당(Colab 미국시간) : h = datetime.now().hour

  # 최근 5일 상승여부에 따라 일자별 5점 (close, volume) 
  def get_5days(feat):
    if feat == 'volume':  # 거래량의 경우, 당일은 일중 시간 보정 필요
      df['volume'].iloc[-1] = df['volume'].iloc[-1] / h * 24
    ser = df[feat] - df[feat].shift()
    res = np.where(ser > 0, 1, 0)[-5:]
    #print(res)
    return res.sum()
        
  #전일가격범위 대비 하여 시초가 가격이 오른 비율
  def get_range():
    range = (df.iloc[-2]['high'] - df.iloc[-2]['low'])
    #print(df.iloc[-2]['close'], range, df['close'][-1]) 
 
    today_range = ( df['close'].iloc[-1] - df['open'].iloc[-1] ) / range
    #print(today_range)

    #전일가격범위 대비 하여 시초가 가격이 오른 비율 
    try: 
      today_range = ( df['close'].iloc[-1] - df['open'].iloc[-1] ) / range

      if today_range < 0.1:
        range_score = 0
      elif today_range < 0.2:
        range_score = 1
      elif today_range < 0.3:
        range_score = 2
      elif today_range < 0.4:
        range_score = 3
      elif today_range < 0.5:
        range_score = 4
      elif today_range >= 0.5:
        range_score = 5
    except:
      print("가격범위 산정 오류(0값 부여) : ", ticker)
      range_score = 0

    return range_score

  def get_trade():
    return df['value'][-1]

  price_score = get_5days('close')
  volume_score = get_5days('volume')
  range_score = get_range()
  trade_score = get_trade()

  return price_score, volume_score, range_score, trade_score 

In [11]:
tickers = pyupbit.get_tickers(fiat="KRW")

coin_score = []
for ticker in tickers:
  try:
    p, v, r, t = get_coin_score(ticker)
    coin_score.append([ticker, p, v, r, t])
  except:
    print("실행오류 : ", ticker)
    pass 
  time.sleep(0.5)

In [21]:
res = pd.DataFrame(coin_score)
res.columns = ['ticker', 'price', 'volume', 'range', 'trade']

# 거래금액 상대 순위 점수 
res['trade'] = (res['trade'].rank(pct=True) * 5).astype(int) 

# 총점 
res['total_score'] = res.iloc[:,1:].sum(axis=1)

# 최종 결과 파일
score_final = res.sort_values(by='total_score', ascending=False).reset_index(drop=True)

In [22]:
# 상위 10종목
score_final.iloc[:10]

,ticker,price,volume,range,trade,total_score
0,KRW-AAVE,2,4,4,4,14
1,KRW-ORBS,4,1,4,4,13
2,KRW-KNC,4,3,2,4,13
3,KRW-SOL,3,1,5,4,13
4,KRW-ANKR,2,3,5,3,13
5,KRW-AERGO,2,2,5,4,13
6,KRW-DOGE,2,3,2,5,12
7,KRW-ETH,2,2,3,4,11
8,KRW-MTL,3,2,2,4,11
9,KRW-TRX,2,1,3,4,10


In [23]:
# 하위 10종목 
score_final.iloc[-10:]

,ticker,price,volume,range,trade,total_score
103,KRW-STRAX,1,1,1,0,3
104,KRW-IQ,1,2,0,0,3
105,KRW-CBK,1,2,0,0,3
106,KRW-GRS,1,0,2,0,3
107,KRW-AHT,1,1,1,0,3
108,KRW-DKA,1,2,0,0,3
109,KRW-SBD,0,1,0,1,2
110,KRW-GAS,0,1,0,1,2
111,KRW-BTT,0,2,0,0,2
112,KRW-STPT,1,1,0,0,2
